In [1]:
import cv2
import numpy as np
import zipfile, os

In [2]:
local_zip = 'hijaiyah_Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [14]:
dir = '/tmp/dataset8020'

In [35]:
traindir = '/tmp/dataset8020/train'
label = []
for subfolder in os.listdir(traindir):
    for image in os.listdir(os.path.join(traindir, subfolder)):
        label.append(subfolder)

In [36]:
classes = {'1.Alif':0,
            '2.Ba': 1,
            '3.ta': 2,
            '4.sa_':3 ,
            '5.ja': 4,
            '6.kha':5 ,
            '7.kho': 6,
            '8.da': 7,
            '9.za_':8 ,
            '10.ro': 9,
            '11.za': 10,
            '12.sa': 11,
            '13.sya':12,
            '14.syo': 13,
            '15.do': 14,
            '16.to': 15,
            '17.zo': 16,
            '18.ain':17, 
            '19.go': 18,
            '20.fa': 19,
            '21.qo': 20,
            '22.ka': 21,
            '23.la': 22,
            '24.ma': 23,
            '25.na': 24,
            '26.wa': 25,
            '27.ha': 26,
            '28.ya': 27}

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split=0.2)

In [49]:


train_dataset = train_datagen.flow_from_directory(batch_size=32,
                                                 directory='/tmp/dataset8020/train',
                                                 shuffle=True,
                                                 target_size=(128, 128), 
                                                 subset="training",
                                                  color_mode ="grayscale",
                                                 class_mode='categorical',
                                                 classes = classes
                                                 )

validation_dataset = train_datagen.flow_from_directory(batch_size=32,
                                                 directory='/tmp/dataset8020/train',
                                                 shuffle=True,
                                                 target_size=(128, 128), 
                                                 subset="validation",
                                                 color_mode ="grayscale",
                                                 class_mode='categorical',
                                                 classes = classes
                                                 )

Found 5376 images belonging to 28 classes.
Found 1344 images belonging to 28 classes.


In [65]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (128, 128, 1)),
    tf.keras.layers.AveragePooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.AveragePooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(28, activation = 'softmax')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 average_pooling2d (AverageP  (None, 63, 63, 32)       0         
 ooling2D)                                                       
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 average_pooling2d_1 (Averag  (None, 30, 30, 64)       0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 57600)             0         
                                                                 
 dense_12 (Dense)            (None, 128)              

In [66]:


# callback membantu proses training agar lebih efisien, mulai dari early stop, check point, sampai dengan custom callback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from keras.callbacks import EarlyStopping


# modelFolder = 'model[0]/' #merupakan folder model yang menggunakan arsitektur neural network squensial,
modelFolder = 'model[5]/' # merupakan folder model yang mengunakan arsitektur inception neural network v2

# checkpoint ini memungkinkan weight disimpan pada saat loss training mencapai titik minimum
checkpointLoss = ModelCheckpoint(f"{modelFolder}bestLoss.hdf5", 
                            monitor='loss', 
                            verbose=1,
                            save_best_only=True, 
                            mode='auto')

# checkpoint ini memungkinkan weight disimpan pada saat loss training pada tahap validation mencapai titik minimum
checkpointValLoss = ModelCheckpoint(f"{modelFolder}bestValLoss.hdf5", 
                            monitor='val_loss', 
                            verbose=1,
                            save_best_only=True, 
                            mode='auto')

# checkpoint early stop memungkinkan model untuk langsung berhenti melakukan proses training ketika
# tidak terdapat penurunan loss (semakin kecil locc semakin bagus). 
# patience = 10, menandakan model akan berhenti ketika tidak ada lagi minimum loss yang didapatkan selama 10 epochs berturut turut
earlyStopVal = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
earlyStop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

# callback khusus yang akan langsung memberhentikan model ketika sudah mencapain akurasi dan loss yang cukup, yakni 95% akurasi dengan loss 0.001
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95 and logs.get('loss')<0.001):
            print("\nAkurasi telah mencapai > 95%! dan loss < 0.001")
            self.model.stop_training = True

callbacks = myCallback()

In [68]:


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
              

In [71]:
epoch = 1000
model.fit(
    train_dataset,
    steps_per_epoch=100,
    epochs= epoch,
    validation_data=validation_dataset,
    validation_steps=5,
    callbacks = [callbacks, checkpointValLoss,earlyStopVal ]

    )

Epoch 1/1000
100/100 [==============================] - ETA: 0s - loss: 2.6816 - accuracy: 0.2003
Epoch 1: val_loss improved from 3.10213 to 2.86527, saving model to model[5]\bestValLoss.hdf5
100/100 [==============================] - 131s 1s/step - loss: 2.6816 - accuracy: 0.2003 - val_loss: 2.8653 - val_accuracy: 0.2000
Epoch 2/1000
100/100 [==============================] - ETA: 0s - loss: 2.4389 - accuracy: 0.2612
Epoch 2: val_loss improved from 2.86527 to 2.79656, saving model to model[5]\bestValLoss.hdf5
100/100 [==============================] - 127s 1s/step - loss: 2.4389 - accuracy: 0.2612 - val_loss: 2.7966 - val_accuracy: 0.2062
Epoch 3/1000
100/100 [==============================] - ETA: 0s - loss: 2.2587 - accuracy: 0.3175
Epoch 3: val_loss improved from 2.79656 to 2.63621, saving model to model[5]\bestValLoss.hdf5
100/100 [==============================] - 127s 1s/step - loss: 2.2587 - accuracy: 0.3175 - val_loss: 2.6362 - val_accuracy: 0.2250
Epoch 4/1000
100/100 [======

In [117]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
test_image = image.load_img('alif.png', target_size=(128, 128), color_mode='grayscale')
test_img = image.img_to_array(test_image)
test_img.shape

(128, 128, 1)

In [118]:
test_img = np.expand_dims(test_img, axis=0)
images = np.vstack([test_img])

In [122]:
predict = model.predict(test_img)
y_true = np.argmax(predict)
result = list(classes.keys())[list(classes.values()).index(y_true)]
print('ini adalah '+ result)

ini adalah 1.Alif
